In [198]:
import pandas as pd

In [199]:
man_df = pd.read_csv('data/caesar_fr.csv', encoding='utf-8', sep=';')
man_df=man_df.sample(frac=1)
caesar_male = man_df[man_df['sex'] == 1]
caesar_male = caesar_male.drop(columns=["sex"], errors='ignore')

In [200]:
caesar_male=caesar_male.loc[:, (caesar_male != 0).any()]

In [201]:
import pandas as pd
import numpy as np

def remplacer_nan_par_moyenne(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            moyenne = df[col].mean(skipna=True)
            df[col] = df[col].fillna(moyenne)
    return df

caesar_malecaesar_malecaesar_male = remplacer_nan_par_moyenne(caesar_male)
caesar_male.head()

,taille,age,weight,tour_de_cheville,longueur_du_bras,tour_de_poitrine,tour_de_poitrine_mesure_a_la_main,hauteur_de_poitrine,hauteur_d_entrejambe,hauteur_des_hanches,hauteur_des_genoux,tour_du_cou,largeur_d_epaule,hauteur_des_epaules,largeur_des_epaules_a_l_horizontales,longueur_de_la_colonne_vertebrale_jusqu_au_poignet,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille,longueur_d_avant_bras
446,167.1066,65.0,68.719188,26.7970,58.5978,96.0882,NaN,120.1928,74.1934,85.6996,46.4058,44.3992,37.9222,136.2964,43.6880,53.1495,53.1114,85.2932,97.3074,22.9489
1300,176.7078,34.0,94.501357,28.3972,62.5094,112.9030,NaN,128.9558,81.0006,86.7918,48.8569,53.8988,39.5732,146.2151,48.4124,58.8518,63.6016,99.2886,103.0986,25.4254
2199,180.8988,49.0,97.975872,28.4988,64.7954,117.9068,NaN,130.6068,78.3082,87.6046,49.7078,46.1010,42.8244,148.6916,54.0004,58.8010,62.7888,94.8944,104.5972,26.4795
3253,175.5902,33.0,78.017824,25.7048,65.4050,101.4984,NaN,128.1938,80.5942,85.9028,48.5013,42.6974,42.2656,144.6530,49.4030,55.3339,58.9026,85.2932,103.0986,25.8064
1068,174.0916,57.0,92.532768,27.1018,61.1124,113.4110,NaN,124.0028,78.5114,90.5002,48.9966,45.8978,42.2148,143.4973,48.9966,54.7116,59.4106,98.6028,101.3968,25.3111


In [202]:
caesar_male["tour_de_cuisse_ratio"] = caesar_male["tour_de_cuisse"] / caesar_male["taille"]
caesar_male["tour_de_taille_ratio"] = caesar_male["tour_de_taille"] / caesar_male["taille"]
caesar_male["largeur_d_epaule_ratio"] = caesar_male["largeur_d_epaule"] / caesar_male["taille"]

In [203]:
caesar_male["tour_de_cuisse_ratio"].describe() 


count    2107.000000
mean        0.333812
std         0.031655
min         0.248715
25%         0.312557
50%         0.331693
75%         0.350709
max         0.536801
Name: tour_de_cuisse_ratio, dtype: float64

In [204]:
def classifier_morphologie(row):
    # Extraction des mesures
    #tour_ventre = row['tour_de_ventre_maximal']
    tour_taille = row['tour_de_taille_ratio']
    forme_poitrine = row['largeur_d_epaule_ratio']
    tour_cuisse = row['tour_de_cuisse_ratio']
    
    
    # --- Catégorie VENTRE (basée sur le ratio ventre/taille) ---
    if tour_taille < 0.462864:
        ventre = "plat"
    elif 0.462864 <= tour_taille<0.536128:
        ventre = "moyen"
    else:
        ventre = "rond"
    
    # --- Catégorie TORSE (basée sur le tour de poitrine seul ou comparé à la taille) ---
    if forme_poitrine < 0.224841:
        torse = "fin"
    elif 0.224841 <=forme_poitrine < 0.242234 :
        torse = "moyen"
    else:
        torse = "large"
    
    # --- Catégorie CUISSES (ratio cuisse / hanches) ---
    if tour_cuisse < 0.312557:
        cuisses = "fines"
    elif  0.312557 <=tour_cuisse<0.350709 :
        cuisses = "moyennes"
    else:
        cuisses = "larges"
    
    return pd.Series([ventre, torse, cuisses], 
                     index=['categorie_ventre', 'categorie_torse', 'categorie_cuisses'])
    
# Remplace `df` par le nom de ton DataFrame
caesar_male[['categorie_ventre', 'categorie_torse', 'categorie_cuisses']] = caesar_male.apply(classifier_morphologie, axis=1)

In [205]:
from sklearn.preprocessing import RobustScaler

# Colonnes à exclure
cols_to_exclude = ["taille","age","weight","categorie_ventre", "categorie_torse", "categorie_cuisses"]

# Colonnes à scaler (tout sauf celles à exclure)

caesar_male = caesar_male.dropna(how='all', axis=1)
cols_to_scale = [col for col in caesar_male.columns if col not in cols_to_exclude]
# On copie le DataFrame pour garder l'original intact
df_scaled = caesar_male.copy()

# On applique RobustScaler uniquement sur les colonnes numériques à scaler
scaler = RobustScaler()
df_scaled[cols_to_scale] = scaler.fit_transform(df_scaled[cols_to_scale])

# Résultat : df_scaled contient les colonnes scalées (sauf celles exclues)


In [206]:
# Supprime toutes les colonnes contenant "ratio" dans leur nom
colonnes_ratio = [col for col in df_scaled.columns if "ratio" in col]
df_scaled.drop(columns=colonnes_ratio, inplace=True)

print(f"✅ Colonnes supprimées : {colonnes_ratio}")

✅ Colonnes supprimées : ['tour_de_cuisse_ratio', 'tour_de_taille_ratio', 'largeur_d_epaule_ratio']


In [207]:
df_scaled.head()

,taille,age,weight,tour_de_cheville,longueur_du_bras,tour_de_poitrine,hauteur_de_poitrine,hauteur_d_entrejambe,hauteur_des_hanches,hauteur_des_genoux,...,hauteur_des_epaules,largeur_des_epaules_a_l_horizontales,longueur_de_la_colonne_vertebrale_jusqu_au_poignet,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille,longueur_d_avant_bras,categorie_ventre,categorie_torse,categorie_cuisses
446,167.1066,65.0,68.719188,0.101266,-1.176796,-0.326296,-0.975182,-0.897959,-0.304348,-0.775835,...,-1.002004,-0.927374,-1.262673,-0.797101,-0.229323,-0.856678,-1.660494,moyen,moyen,moyennes
1300,176.7078,34.0,94.501357,0.898734,-0.325967,0.944338,0.032117,0.013605,-0.160535,-0.162162,...,0.041416,0.111732,0.116743,0.699275,0.806391,-0.114007,-0.456790,rond,fin,larges
2199,180.8988,49.0,97.975872,0.949367,0.171271,1.322457,0.221898,-0.346939,-0.053512,0.050874,...,0.301937,1.340782,0.104455,0.583333,0.481203,0.078176,0.055556,moyen,moyen,moyennes
3253,175.5902,33.0,78.017824,-0.443038,0.303867,0.082534,-0.055474,-0.040816,-0.277592,-0.251192,...,-0.122912,0.329609,-0.734255,0.028986,-0.229323,-0.114007,-0.271605,moyen,moyen,moyennes
1068,174.0916,57.0,92.532768,0.253165,-0.629834,0.982726,-0.537226,-0.319728,0.327759,-0.127186,...,-0.244489,0.240223,-0.884793,0.101449,0.755639,-0.332248,-0.512346,rond,large,moyennes


In [208]:
df_scaled.columns

Index(['taille', 'age', 'weight', 'tour_de_cheville', 'longueur_du_bras',
       'tour_de_poitrine', 'hauteur_de_poitrine', 'hauteur_d_entrejambe',
       'hauteur_des_hanches', 'hauteur_des_genoux', 'tour_du_cou',
       'largeur_d_epaule', 'hauteur_des_epaules',
       'largeur_des_epaules_a_l_horizontales',
       'longueur_de_la_colonne_vertebrale_jusqu_au_poignet', 'tour_de_cuisse',
       'tour_de_taille', 'hauteur_de_la_taille', 'longueur_d_avant_bras',
       'categorie_ventre', 'categorie_torse', 'categorie_cuisses'],
      dtype='object')

In [209]:
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib


X_train=df_scaled[['taille', 'age', 'weight',
              'categorie_ventre', 'categorie_torse', 'categorie_cuisses']]

# Colonnes numériques et catégorielles
cols_num = ["taille","age","weight"]
cols_cat = ["categorie_ventre", "categorie_torse", "categorie_cuisses"]

# Pipeline pour les colonnes numériques : imputation par médiane + scaling
pipeline_num = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

# Pipeline pour les colonnes catégorielles : imputation par la valeur la plus fréquente + one-hot
pipeline_cat = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline globale de prétraitement
preprocessor = ColumnTransformer(transformers=[
    ("num", pipeline_num, cols_num),
    ("cat", pipeline_cat, cols_cat)
])

# Fit sur X_train (tu dois définir X_train comme DataFrame brut)
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['taille', 'age', 'weight']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['categorie_ventre', 'categorie_torse',
                                  'categorie_cuisses'])])

In [210]:
print("Colonnes dans X_train :", X_train.columns.tolist())
print("Colonnes numériques :", cols_num)
print("Colonnes catégorielles :", cols_cat)

Colonnes dans X_train : ['taille', 'age', 'weight', 'categorie_ventre', 'categorie_torse', 'categorie_cuisses']
Colonnes numériques : ['taille', 'age', 'weight']
Colonnes catégorielles : ['categorie_ventre', 'categorie_torse', 'categorie_cuisses']


In [211]:
import joblib
import os

# 📁 Répertoire contenant les fichiers .pkl
dossier_modeles = "./modeles_tpot"  # modifie ce chemin si nécessaire

# 📌 Chargement de tous les modèles
modeles = {}
for nom_fichier in os.listdir(dossier_modeles):
    if nom_fichier.endswith(".pkl"):
        nom_cible = nom_fichier.replace("tpot_", "").replace(".pkl", "")
        chemin_complet = os.path.join(dossier_modeles, nom_fichier)
        modeles[nom_cible] = joblib.load(chemin_complet)

print(f"✅ {len(modeles)} modèles chargés :", list(modeles.keys()))

✅ 16 modèles chargés : ['hauteur_des_epaules', 'hauteur_des_genoux', 'hauteur_des_hanches', 'hauteur_de_la_taille', 'hauteur_de_poitrine', 'hauteur_d_entrejambe', 'largeur_des_epaules_a_l_horizontales', 'largeur_d_epaule', 'longueur_de_la_colonne_vertebrale_jusqu_au_poignet', 'longueur_du_bras', 'longueur_d_avant_bras', 'tour_de_cheville', 'tour_de_cuisse', 'tour_de_poitrine', 'tour_de_taille', 'tour_du_cou']


In [212]:
import os

dossier_modeles = "./modeles_tpot"  # adapte ce chemin
noms_fichiers = [f for f in os.listdir(dossier_modeles) if f.endswith(".pkl")]

# Dossier de sauvegarde des pipelines complets
dossier_output = "pipelines_complets"
os.makedirs(dossier_output, exist_ok=True)

for nom_fichier in noms_fichiers:
    chemin_modele = os.path.join(dossier_modeles, nom_fichier)
    
    # Charger le modèle TPOT
    tpot_model = joblib.load(chemin_modele)
    
    # Créer le pipeline complet
    full_pipeline = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", tpot_model)
    ])
    
    # Sauvegarder le pipeline complet
    nom_fichier_pipeline = f"pipeline_{nom_fichier}"
    chemin_output = os.path.join(dossier_output, nom_fichier_pipeline)
    joblib.dump(full_pipeline, chemin_output)

    print(f"✔ Pipeline sauvegardé : {chemin_output}")

✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_des_epaules.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_des_genoux.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_des_hanches.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_de_la_taille.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_de_poitrine.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_hauteur_d_entrejambe.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_des_epaules_a_l_horizontales.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_largeur_d_epaule.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_longueur_de_la_colonne_vertebrale_jusqu_au_poignet.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_longueur_du_bras.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_longueur_d_avant_bras.pkl
✔ Pipeline sauvegardé : pipelines_complets\pipeline_tpot_tour_de

In [213]:
import os
import joblib
import pandas as pd

# 📁 1. Dossier où se trouvent les 22 pipelines
dossier_pipelines = "pipelines_complets"  # adapte selon ton cas

# 🧠 2. Charger tous les pipelines .pkl
pipelines = {}
for filename in os.listdir(dossier_pipelines):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        nom_variable = filename.replace("pipeline_tpot_", "").replace(".pkl", "")
        pipelines[nom_variable] = joblib.load(path)

# 📦 3. Exemple de nouvelles données
new_data = pd.DataFrame([{
    "taille": 187,
    "weight": 80,
    "age": 33,
    "categorie_ventre": "moyen",
    "categorie_cuisses": "medium",
    "categorie_torse": "large"
}])

# 🧮 4. Appliquer chaque pipeline à la même ligne de données
predictions = {}
for nom_variable, pipeline in pipelines.items():
    y_pred = pipeline.predict(new_data)
    predictions[nom_variable] = y_pred[0]  # extraire la valeur unique

# 🖨️ 5. Résultat
print("Prédictions :")
for var, val in predictions.items():
    print(f"{var} = {val:.2f}")


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset

Prédictions :
hauteur_des_epaules = 151.35
hauteur_des_genoux = 52.07
hauteur_des_hanches = 93.48
hauteur_de_la_taille = 110.91
hauteur_de_poitrine = 134.75
hauteur_d_entrejambe = 86.02
largeur_des_epaules_a_l_horizontales = 43.20
largeur_d_epaule = 43.45
longueur_de_la_colonne_vertebrale_jusqu_au_poignet = 60.93
longueur_du_bras = 66.94
longueur_d_avant_bras = 10.89
tour_de_cheville = 27.35
tour_de_cuisse = 59.78
tour_de_poitrine = 96.97
tour_de_taille = 82.57
tour_du_cou = 41.31


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
